In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import math

from qiskit.visualization import plot_histogram 

In [6]:
# ex. # of partition, partition qubit ([[0,1], [2,3], [7,8], [9,10]]), condition qubit ([4,5,6])
def my_marginal(data, p_q, c_q, qubits, device_type, wh):

    total_partition = len(p_q)
    reduced_q = len(p_q[0])
    partition_q = len(c_q)
    
    ret_p = []
    ret_c = []
    for i in data.iterrows():

        dic_p = [{} for i in range(total_partition)]
        for l in range(total_partition):
            for j in range(2 ** partition_q ):
                bit = str(bin(j)[2:]).zfill(1)
                dic_p[l][bit] = {}
                for k in range(2 ** reduced_q):
                    bit_2 = str(bin(k)[2:]).zfill(reduced_q)
                    dic_p[l][bit][bit_2] = 0
        
        dic_c = {}
        for j in range(2 ** partition_q):
            bit = str(bin(j)[2:]).zfill(1)
            dic_c[bit] = 0
                
        
        for j, k in zip(i[1], range(len(i[1]))):
            bit = str(bin(k)[2:]).zfill(qubits)   
            selected_chars_c = [bit[qubits - 1 - m] for m in c_q]
            new_string_c = ''.join(selected_chars_c)
            
            for l in range(total_partition):

                selected_chars = [bit[qubits - 1 - m] for m in p_q[l]]
                new_string = ''.join(selected_chars[::-1])
                dic_p[l][new_string_c][new_string] += j

            dic_c[new_string_c] += j            
            
        ret_p.append(dic_p)
        ret_c.append(dic_c)
        
    a = [[] for i in range(total_partition * 2 ** partition_q) ]
    for i in ret_p:
        for j in range(len(i)):
            for k in range(len(i[j])):
                bit = str(bin(k)[2:]).zfill(1)
                a[k + j * len(i)].append(i[j][bit])
    
    res_c_df = pd.DataFrame(ret_c)
    res_c_df.to_csv('./' + device_type + '/' + device_type + '_' + str(qubits) +  'qubits' + wh + 'D' + '.csv', index=False, header=True)
    
    for l in range(len(a)):
        test_df = pd.DataFrame(a[l])
        test_df
        for i, j, k in zip(range(len(test_df)), test_df.iterrows(), res_c_df.iterrows()):
            test_df.iloc[i] = j[1] / k[1].values[l % (2 ** partition_q)]
        test_df.to_csv('./' + device_type + '/' + device_type + '_' + str(qubits) +  'qubits' + wh + str(l) + '.csv', index=False, header=True)

    return ret_p, ret_c
    

In [17]:
#n_qubits = [7]
#n_data = 300
#times = [39] # 11700, 39
#device_type = 'ibm_lagos' 
#number = 0

n_qubits = [7]
n_data = 300
times = [34] # 10200, 34
device_type = 'ibmq_jakarta' 
number = 0

In [18]:
X = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_obs_' + str(number) + '.csv')
Y = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_ideal_' + str(number) + '.csv')
LI = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_miti_' + str(number) + '.csv')

In [19]:
# data, partition qubit, condition qubit, # of qubit, device, name
my_marginal(X, [[0, 1, 2], [4, 5, 6]], [3], 7, device_type, '_obs_CI_')
my_marginal(Y, [[0, 1, 2], [4, 5, 6]], [3], 7, device_type, '_ideal_CI_') 

([[{'0': {'000': 0.044960831,
     '001': 0.049851045999999996,
     '010': 0.026796507999999997,
     '011': 0.029711107,
     '100': 0.244693317,
     '101': 0.271307613,
     '110': 0.145836673,
     '111': 0.161698778},
    '1': {'000': 0.0011597040000000001,
     '001': 0.001285803,
     '010': 0.0006911759999999999,
     '011': 0.000766313,
     '100': 0.006311242999999999,
     '101': 0.006997751,
     '110': 0.003761577,
     '111': 0.004170582}},
   {'0': {'000': 0.00115482,
     '001': 0.000656726,
     '010': 0.111836344,
     '011': 0.06360015,
     '100': 0.005196394,
     '101': 0.002955139,
     '110': 0.5032584999999999,
     '111': 0.2861978},
    '1': {'000': 2.9787e-05,
     '001': 1.6929e-05,
     '010': 0.0028845609999999995,
     '011': 0.001640492,
     '100': 0.00013398999999999998,
     '101': 7.614e-05,
     '110': 0.012980421999999998,
     '111': 0.007381828}}],
  [{'0': {'000': 0.14589735,
     '001': 0.039731216,
     '010': 0.09759342599999998,
     '011'

In [139]:
# ex) # of partition, partition qubit ([[0,1], [2,3], [7,8], [9,10]]), condition ([4,5,6])
def my_marginal(data, p_q, c_q, qubits, device_type, wh):

    total_partition = len(p_q)
    reduced_q = len(p_q[0])
    partition_q = len(c_q)
    
    ret_p = []
    ret_c = []
    for i in data.iterrows():

        dic_p = [{} for i in range(total_partition)]
        for l in range(total_partition):
            for j in range(2 ** partition_q ):
                bit = str(bin(j)[2:]).zfill(1)
                dic_p[l][bit] = {}
                for k in range(2 ** reduced_q):
                    bit_2 = str(bin(k)[2:]).zfill(reduced_q)
                    dic_p[l][bit][bit_2] = 0
        
        dic_c = {}
        for j in range(2 ** partition_q):
            bit = str(bin(j)[2:]).zfill(1)
            dic_c[bit] = 0
                
        
        for j, k in zip(i[1], range(len(i[1]))):
            bit = str(bin(k)[2:]).zfill(qubits)   
            selected_chars_c = [bit[qubits - 1 - m] for m in c_q]
            new_string_c = ''.join(selected_chars_c)
            
            for l in range(total_partition):

                selected_chars = [bit[qubits - 1 - m] for m in p_q[l]]
                new_string = ''.join(selected_chars[::-1])
                dic_p[l][new_string_c][new_string] += j

            dic_c[new_string_c] += j            
            
        ret_p.append(dic_p)
        ret_c.append(dic_c)
        
    a = [[] for i in range(total_partition * 2 ** partition_q) ]
    for i in res_p:
        for j in range(len(i)):
            for k in range(len(i[j])):
                bit = str(bin(k)[2:]).zfill(1)
                a[k + j * len(i)].append(i[j][bit])
    
    res_c_df = pd.DataFrame(res_c)
    res_c_df.to_csv('./' + device_type + '/' + device_type + '_' + str(qubits) +  'qubits' + wh + 'D' + '.csv', index=False, header=True)
    
    for l in range(len(a)):
        test_df = pd.DataFrame(a[l])
        test_df
        for i, j, k in zip(range(len(test_df)), test_df.iterrows(), res_c_df.iterrows()):
            test_df.iloc[i] = j[1] / k[1].values[l % (2 ** partition_q)]
        test_df.to_csv('./' + device_type + '/' + device_type + '_' + str(qubits) +  'qubits' + wh + str(l) + '.csv', index=False, header=True)

    return ret_p, ret_c

res_p, res_c = my_marginal(X, [[0, 1, 2], [4, 5, 6]], [3], 7, device_type, '_obs_CI_')
    